# Assignment #6 - Data Gathering and Warehousing - DSSA-5102

Instructor: Melissa Laurino</br>
Spring 2024</br>

Name: Colin Glory
</br>
Date: 2/22/24
<br>
<br>
**At this time in the semester:** <br>
--We have explored a dataset. <br>
--We have cleaned our dataset. <br>
--We created a Github account with a repository for this class and included a Metadata file about our data. <br>
--We introduced general SQL syntax, queries, and applications in R.<br>
<br>

Now we will start the process of uploading our dataset into a database. There are many different ways to upload your .csv data into a database (.db file). Databases can be created in many open source applications, MySQL workbench, and even some websites can load your .csv data into a database...for a small fee. Instead of using an application, we are going to first create our database for our dataset from scratch in R. On a much larger scale, data may be automatically uploaded to a database was once it is aquired.<br>

There are two R packages below we will use to create two separate databases for practice. One will be stored on our MySQL server using R package RMariaDB, and the other will be stored locally using R package SQLite. <br>
<br>
Follow the instructions below to complete the assignment. Answer any questions in markdown cell boxes. Be sure to comment all code.


### Creating our database from scratch to integrate with MySQL Workbench in R.<br>

In [6]:
#Install necessary packages. After you have them installed, do not install them again. You should already have these installed from Assignment #4.
#Continuing to run install.packages could update the versions of other dependent packages.
#install.packages("RMariaDB")
install.packages("DBI")
install.packages("sqldf")
#install.packages("RSQLite")

Warning message:
"package 'DBI' is in use and will not be installed"
Installing package into 'C:/Users/cglor/AppData/Local/R/win-library/4.3'
(as 'lib' is unspecified)

also installing the dependencies 'gsubfn', 'proto', 'chron'




package 'gsubfn' successfully unpacked and MD5 sums checked
package 'proto' successfully unpacked and MD5 sums checked
package 'chron' successfully unpacked and MD5 sums checked
package 'sqldf' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\cglor\AppData\Local\Temp\RtmpAX5POt\downloaded_packages


### Creating a database from scratch in R using library(RMariaDB)<br>

https://cran.r-project.org/web/packages/RMariaDB/index.html

**BEFORE YOU BEGIN!**<br>
Is your MySQL Server running on your local machine?<br>
**Start the server** if it is not running already.

In [12]:
#Create a database from scratch using library RMariaDB 
library(RMariaDB)
library(DBI)

In [9]:
#Define our variables. We set these during our first class in our technology set up. 
#If you are unsure of these variables, do not guess. 
#Visit MySQL Workbench for the localport number, host and user.
localuserpassword <- "Charlie@bowie212" #We wrote this password down in our first class.
localport <- 3306 #This is your local instance number when you open MySQL Workbench.
db_name <- "assignment6" #Since we are creating our database from scratch, we most likely 
#do not have a name yet. This will be the name when the database is created.

In [13]:
#Connect to the server using MariaDB()
con <- dbConnect(RMariaDB::MariaDB(), 
                 host = 'localhost',
                 user = 'root',
                 password = localuserpassword,
                 port = localport)
#In order to connect to the server, we must include all of the above.

In [16]:
#Create your first database integrated with MySQL Workbench. Use:
dbExecute(con, paste("CREATE DATABASE IF NOT EXISTS ", db_name))
#This code will create the database if it does not already exist with the db_name you defined above.
#Depending on your package versions, you may not need paste().

#If you have multiple databases, it is important to specify what database you are using.
#This especially applies if you have reused code to create multiple databases!
dbExecute(con, paste("USE", db_name))

#Note above paste() function.
#This may not be needed and depends on your version of R/SQLpackages.

[1] 1

[1] 0

**STOP**<br><br>
Open MySQL Workbench.<br>
Under MySQL Connections, click Local Instance<br>
Click the Schemas tab<br>
**You should now see a new (empty) database that you created**<br>
If it does not show up right away, hit refresh (The circular arrows)

In [17]:
#Read in the CLEAN .csv file we will use to populate our database. This is the same dataset
#that you cleaned for Assignment #2.
#A subset of 2014-2023 Sea turtle sightings with Cape May Whale Watch and Research Center
df <- read.csv("cleaned_data.csv")

In [18]:
#Preview the dataframe. 
head(df)

,X,job_title,experience_level,employment_type,work_models,work_year,employee_residence,salary,salary_currency,salary_in_usd,company_location,company_size
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>
1,1,Data Engineer,Mid-level,Full-time,Remote,2024,United States,148100,USD,148100,United States,Medium
2,2,Data Engineer,Mid-level,Full-time,Remote,2024,United States,98700,USD,98700,United States,Medium
3,3,Data Scientist,Senior-level,Full-time,Remote,2024,United States,140032,USD,140032,United States,Medium
4,4,Data Scientist,Senior-level,Full-time,Remote,2024,United States,100022,USD,100022,United States,Medium
5,5,BI Developer,Mid-level,Full-time,On-site,2024,United States,120000,USD,120000,United States,Medium
6,6,BI Developer,Mid-level,Full-time,On-site,2024,United States,62100,USD,62100,United States,Medium


In [22]:
#After previewing the dataframe, complete any extra data cleaning steps that may be needed.
#Read the date column as DATE, Read the time column as TIME, etc.
names(df)[1] <- "employee_ID"



#Double check the new date and time format in a quick preview.
head(df)

,employee_ID,job_title,experience_level,employment_type,work_models,work_year,employee_residence,salary,salary_currency,salary_in_usd,company_location,company_size
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>
1,1,Data Engineer,Mid-level,Full-time,Remote,2024,United States,148100,USD,148100,United States,Medium
2,2,Data Engineer,Mid-level,Full-time,Remote,2024,United States,98700,USD,98700,United States,Medium
3,3,Data Scientist,Senior-level,Full-time,Remote,2024,United States,140032,USD,140032,United States,Medium
4,4,Data Scientist,Senior-level,Full-time,Remote,2024,United States,100022,USD,100022,United States,Medium
5,5,BI Developer,Mid-level,Full-time,On-site,2024,United States,120000,USD,120000,United States,Medium
6,6,BI Developer,Mid-level,Full-time,On-site,2024,United States,62100,USD,62100,United States,Medium


In [45]:
#What are all of the column names for our dataset? It is important to know the column names
#from the .csv because these are the field names we will want to use for our first table.
#Remember, the field names represent the column names of the csv/table.
colnames(df)
head(df)

[1] "employee_ID"        "job_title"          "experience_level"  
 [4] "employment_type"    "work_models"        "work_year"         
 [7] "employee_residence" "salary"             "salary_currency"   
[10] "salary_in_usd"      "company_location"   "company_size"

,employee_ID,job_title,experience_level,employment_type,work_models,work_year,employee_residence,salary,salary_currency,salary_in_usd,company_location,company_size
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>
1,1,Data Engineer,Mid-level,Full-time,Remote,2024,United States,148100,USD,148100,United States,Medium
2,2,Data Engineer,Mid-level,Full-time,Remote,2024,United States,98700,USD,98700,United States,Medium
3,3,Data Scientist,Senior-level,Full-time,Remote,2024,United States,140032,USD,140032,United States,Medium
4,4,Data Scientist,Senior-level,Full-time,Remote,2024,United States,100022,USD,100022,United States,Medium
5,5,BI Developer,Mid-level,Full-time,On-site,2024,United States,120000,USD,120000,United States,Medium
6,6,BI Developer,Mid-level,Full-time,On-site,2024,United States,62100,USD,62100,United States,Medium


We can use a new SQL statement CREATE TABLE to create our first table in our new database.<br>
Everyone's data is different! Choose the SQL data types that fit YOUR data needs!<br>
SQL Data Types: https://www.w3schools.com/sql/sql_datatypes.asp

In [40]:
#Create our first table in the database file using SQL statements:
#We want our table column names to match what is in the .csv file
#Every table will be different depending on your dataset.
first_table_query <- "CREATE TABLE Employee (
                        employee_ID INT PRIMARY KEY,
                        job_title VARCHAR(255),
                        experience_level VARCHAR(255),
                        employment_type VARCHAR(255),
                        work_models VARCHAR(255),
                        work_year INT,
                        employee_residence VARCHAR(255),
                        company_location VARCHAR(255),
                        company_size VARCHAR(255)
                                       
                                              );"
#Execute the query:
dbExecute(con, first_table_query)

[1] 0

Define and list your SQL data types for your first table: <br><br>
**My SQL data types for my first table, table_name:**<br>

X








Why did you choose these values to make up your first database table? What did you choose for your primary key and why?

In [41]:
#There are multiple ways to populate the fields of the table. 
#Another option is to add a subset of the data into data table, and then populate the database table.
#Please feel free to change or alter the code below.

#This is our table name we created
table_name <- "Employee"

#Write a query that will show the column names from the table we created
query <- paste("SHOW COLUMNS FROM", table_name)

#Get the query and retrieve the column/field names from the database(con)
db_columns <- dbGetQuery(con, query)$Field

#Define the common columns between the CSV data (Column names) and the database (Table fields)
common_columns <- intersect(colnames(df), db_columns)

In [42]:
#These are the ONLY columns that we will transfer into the table.
common_columns

[1] "employee_ID"        "job_title"          "experience_level"  
[4] "employment_type"    "work_models"        "work_year"         
[7] "employee_residence" "company_location"   "company_size"

In [43]:
#Create a subset of the original .csv data turtles with only the specified column names.
df_subset <- df[, common_columns]

In [46]:
#Write the (table) subset of data to the species table in the database class_example
#Append = TRUE because we WANT to populate the table!
#Since we have already specified the field names, we do not need to redefine them = FALSE
#dbWriteTable(con, "Employee", df_subset, append = TRUE, row.names = FALSE)

# Write the (table) subset of data to the 'Employee' table in the database
# Specify NULL for the employee_ID column to allow auto-incrementing
dbWriteTable(con, "Employee", df_subset, append = TRUE, row.names = FALSE, overwrite = FALSE, null = "NULL")


Another option in creating tables could be to first create the table in R. We could select for certain columns and format them as.data.table() and then populate them into the SQL tables.

**STOP**<br><br>
In MySQL Workbench, you should see your new table that you have created and populated.<br>
You can run an SQL query directly in MySQL Workbench!<br>
You can also run a query below:

In [55]:
#Now that we have populated our table, let's try out a query.

#Write a query specific to your database:
# Construct the SQL query to select all columns from the 'Employee' table
query1 <- "SELECT * 
           FROM Employee"

# Execute the query
result <- dbGetQuery(con, query1)

# Print the result
print(result)

      employee_ID                                job_title experience_level
1               1                            Data Engineer        Mid-level
2               2                            Data Engineer        Mid-level
3               3                           Data Scientist     Senior-level
4               4                           Data Scientist     Senior-level
5               5                             BI Developer        Mid-level
6               6                             BI Developer        Mid-level
7               7                         Research Analyst      Entry-level
8               8                         Research Analyst      Entry-level
9               9                            Data Engineer  Executive-level
10             10                            Data Engineer  Executive-level
11             11          Business Intelligence Developer        Mid-level
12             12          Business Intelligence Developer        Mid-level
13          

In [57]:
#Write a second query specific to your database:
query2 <- "SELECT employee_ID, job_title
           FROM Employee"

# Execute the query
result <- dbGetQuery(con, query2)

# Print the result
print(result)

      employee_ID                                job_title
1               1                            Data Engineer
2               2                            Data Engineer
3               3                           Data Scientist
4               4                           Data Scientist
5               5                             BI Developer
6               6                             BI Developer
7               7                         Research Analyst
8               8                         Research Analyst
9               9                            Data Engineer
10             10                            Data Engineer
11             11          Business Intelligence Developer
12             12          Business Intelligence Developer
13             13                           Data Scientist
14             14                           Data Scientist
15             15                            Data Engineer
16             16                            Data Engine

**STOP**<br>
To create a new schema diagram for your new database (Even though it only has one table...it's good practice!)<br>
Open MySQL Workbench again<br>
Click Home<br>
Click the Models icon<br>
Click the > icon to the right of "Models"<br>
Choose “Create EER Model from Database” <br>
The Reverse Engineer Database Wizard starts and will walk you through your first database schema diagram.<br>
Save your model. <br>
You can now add relationships and or modify tables...but for this assignment, all we need is that first table. <br>

**Add a screen shot of your first schema diagram (The table) to your repository/Blackboard subission.**

In [58]:
#Close the database connection :)
dbDisconnect(con)

Now what if we wanted to explore the number of each species observed within each separate body of water together? We'll begin JOINing tables in our next assignment.

### Creating our database from scratch locally and explore with MySQL in R.

### Creating a database from scratch in R using library(RSQLite)<br>

https://cran.r-project.org/web/packages/RSQLite/index.html

In [ ]:
#BEFORE YOU BEGIN!
#Is your MySQL Server running on your local machine?
#Doesn't matter this time, please continue! :)

In [59]:
library(DBI)
library(RSQLite)

Warning message:
"package 'RSQLite' was built under R version 4.3.2"


In [88]:
#The only database connection parameters we need here are the name of the database we
#will create and our first table within the database.
db_path <- "data_science_salaries.db"
table_name <- "Employee"

In [89]:
#Connect to the SQLite database 
#With RSQLite, it will create the database FILE if it does not exist.
con <- dbConnect(RSQLite::SQLite(), dbname = db_path)

**STOP HERE**<br>
Before moving on, it is **important** to understand the difference of what we have just completed. Using RSQLite, we have created a database LOCALLY. Notice we did not specify a user, host, or password! We can find this database as a file on our machine. The .db file is created in the same location or working directory you are currently in. If you did not specify a working directory, the .db file is created where this .ipynb is located. 

In [90]:
#We have the option to start from scratch and read in the .csv file again as a new dataframe or continue with the same dataframe from above.

head(df)

,employee_ID,job_title,experience_level,employment_type,work_models,work_year,employee_residence,salary,salary_currency,salary_in_usd,company_location,company_size
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>
1,1,Data Engineer,Mid-level,Full-time,Remote,2024,United States,148100,USD,148100,United States,Medium
2,2,Data Engineer,Mid-level,Full-time,Remote,2024,United States,98700,USD,98700,United States,Medium
3,3,Data Scientist,Senior-level,Full-time,Remote,2024,United States,140032,USD,140032,United States,Medium
4,4,Data Scientist,Senior-level,Full-time,Remote,2024,United States,100022,USD,100022,United States,Medium
5,5,BI Developer,Mid-level,Full-time,On-site,2024,United States,120000,USD,120000,United States,Medium
6,6,BI Developer,Mid-level,Full-time,On-site,2024,United States,62100,USD,62100,United States,Medium


In [81]:
# Get the column names of the dataframe
colnames_df <- colnames(df)

# Subset the CSV data to include only the common columns
df_subset <- df[, colnames_df]



In [82]:
#Worked
head(df)

,employee_ID,job_title,experience_level,employment_type,work_models,work_year,employee_residence,salary,salary_currency,salary_in_usd,company_location,company_size
,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<chr>,<int>,<chr>,<chr>
1,1,Data Engineer,Mid-level,Full-time,Remote,2024,United States,148100,USD,148100,United States,Medium
2,2,Data Engineer,Mid-level,Full-time,Remote,2024,United States,98700,USD,98700,United States,Medium
3,3,Data Scientist,Senior-level,Full-time,Remote,2024,United States,140032,USD,140032,United States,Medium
4,4,Data Scientist,Senior-level,Full-time,Remote,2024,United States,100022,USD,100022,United States,Medium
5,5,BI Developer,Mid-level,Full-time,On-site,2024,United States,120000,USD,120000,United States,Medium
6,6,BI Developer,Mid-level,Full-time,On-site,2024,United States,62100,USD,62100,United States,Medium


In [91]:
# Define the columns you want to keep
columns_for_employee <- c('employee_ID', 'job_title', 'experience_level', 'employment_type', 'work_models', 'work_year', 'employee_residence', 'company_location', 'company_size')

# Generate the SQL statement to create a new table with desired columns
create_new_table_query <- paste("CREATE TABLE IF NOT EXISTS", table_name, "(",
                                paste(columns_for_employee, collapse = ", "),
                                ");")

# Execute the query to create the new table with desired columns
taketwo_first_table <- dbExecute(con, create_new_table_query)


In [94]:
# Subset the original dataframe to include only the desired columns
df_subset2 <- df[, columns_for_employee]

# Populate the table:
dbWriteTable(con, "Employee", df_subset2, append = TRUE, row.names = FALSE, overwrite = FALSE, null = "NULL")

Write a first query for our local database:

In [95]:
#Let's test it:
qu <- dbSendQuery(con, "SELECT *
                        FROM Employee



                            ")
                    
dbFetch(qu)

employee_ID,job_title,experience_level,employment_type,work_models,work_year,employee_residence,company_location,company_size
<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>
1,Data Engineer,Mid-level,Full-time,Remote,2024,United States,United States,Medium
2,Data Engineer,Mid-level,Full-time,Remote,2024,United States,United States,Medium
3,Data Scientist,Senior-level,Full-time,Remote,2024,United States,United States,Medium
4,Data Scientist,Senior-level,Full-time,Remote,2024,United States,United States,Medium
5,BI Developer,Mid-level,Full-time,On-site,2024,United States,United States,Medium
6,BI Developer,Mid-level,Full-time,On-site,2024,United States,United States,Medium
7,Research Analyst,Entry-level,Full-time,On-site,2024,United States,United States,Medium
8,Research Analyst,Entry-level,Full-time,On-site,2024,United States,United States,Medium
9,Data Engineer,Executive-level,Full-time,Remote,2024,United States,United States,Medium


We have successfully queried from our database that we created locally using R library RSQLite.

Write a second query for our local database:

In [98]:
qu <- dbSendQuery(con, "SELECT employee_ID, experience_level
                        FROM Employee



                            ")
                    
dbFetch(qu)







Warning message:
"Closing open result set, pending rows"


employee_ID,experience_level
<int>,<chr>
1,Mid-level
2,Mid-level
3,Senior-level
4,Senior-level
5,Mid-level
6,Mid-level
7,Entry-level
8,Entry-level
9,Executive-level


Write a third query for our local database:

In [99]:
qu <- dbSendQuery(con, "SELECT employee_ID, experience_level, employment_type
                        FROM Employee



                            ")
                    
dbFetch(qu)





Warning message:
"Closing open result set, pending rows"


employee_ID,experience_level,employment_type
<int>,<chr>,<chr>
1,Mid-level,Full-time
2,Mid-level,Full-time
3,Senior-level,Full-time
4,Senior-level,Full-time
5,Mid-level,Full-time
6,Mid-level,Full-time
7,Entry-level,Full-time
8,Entry-level,Full-time
9,Executive-level,Full-time


In [100]:
#Close the database connection :)
dbDisconnect(con)

Warning message in connection_release(conn@ptr):
"There are 1 result in use. The connection will be released when they are closed"
